In [1]:
import os
import sys
sys.path.append('../')  # subir 1 nivel desde notebooks a src
from src.config.settings import COUNTRY, SRC_DIR

In [2]:
from src.sources.inventory.app import Inventory
db_loader = Inventory()
df_data_base = db_loader.load_db_from_country_selected()

CO


c:\Users\JTRUJILLO\Documents\Galgo\Scripts\Otros\deep_research_models\notebooks\..\src\sources\inventory\app.py:23: DtypeWarning: Columns (8,23,30) have mixed types. Specify dtype option on import or set low_memory=False.
  data_base = pd.read_csv(CO_INVENTORY_PATH)


In [3]:
models_list = [
    "CO731-tvs-raider-125",
    "CO928-akt-motos-nkd-125",
    "CO2961-hero-hunk-125-r",
    "CO2861-tvs-raider-fi-125",
    "CO1033-bajaj-boxer-ct-100-es",
    "CO1996-hero-hunk-160r-4v",
    "CO736-tvs-apache-rtr-160-4v-xc-fi",
    "CO734-tvs-apache-rtr-160-4v-xconnect",
    "CO1480-hero-eco-deluxe-cw-c-2024",
    "CO569-tvs-neo-nx",
    # "CO570-tvs-raider-125-racing" # Problemática
]

In [4]:
from src.utils.datetime_utils import get_current_date_format_yyyy_mm_dd
from src.core.base_knowledge import get_basic_data_model
from src.core.prompt_utils import *
from src.core.gemini_processor import GeminiProcessor
gemini_processor = GeminiProcessor()

In [5]:
knowledge_base_final = ""

for idx, model in enumerate(models_list):
    df_modelo = df_data_base[df_data_base["code"] == model]
    data_model = get_basic_data_model(df_modelo, COUNTRY)
    print(f"Probando modelo: {data_model['code']} - {data_model['marca']} {data_model['modelo']}")

    header = f"""=================================================
CODE = {data_model["code"]} | {data_model["marca"]} {data_model["modelo"]}
=================================================
"""

    basic_data_model_info = f""" [DATOS BASICOS]
Marca: {data_model["marca"]}
Modelo: {data_model["modelo"]}
Año: {data_model["año"]}
Pais: {data_model["pais"]}
Tipo: {data_model["tipo"]}
URL: {data_model["url"]}
    """

    footer = f"""================================================="""

    # Leer plantilla del prompt
    prompt_from_file = read_prompt_from_file(f"../src/data/input/prompts/knowledge_base_template.md")


    # Reemplazar variables
    first_research_prompt = replace_variables(prompt_from_file, {
        "{marca}": data_model["marca"],
        "{modelo}": data_model["modelo"],
        "{año}": str(data_model["año"]),
        "{pais}": COUNTRY,
        "{tipo}": data_model["tipo"]
        })

    deep_research = gemini_processor.send_prompt(first_research_prompt)

    # Footer: doble separador entre modelos
    if idx < len(models_list) - 1:  # No es el último modelo
        footer = f"""=================================================
=================================================
"""
    else:  # Es el último modelo
        footer = f"""=================================================
================================================="""

    # Construir knowledge base de este modelo
    knowledge_base = header + "\n" + basic_data_model_info + "\n" + deep_research + "\n" + footer + "\n"
    knowledge_base_final += knowledge_base

Probando modelo: CO731-tvs-raider-125 - TVS Raider 125


c:\Users\JTRUJILLO\Documents\Galgo\Scripts\Otros\deep_research_models\notebooks\..\src\core\gemini_processor.py:9: UserWarning: Interactions usage is experimental and may change in future versions.
  interaction = self.client.interactions.create(


Probando modelo: CO928-akt-motos-nkd-125 - AKT Motos NKD 125
Probando modelo: CO2961-hero-hunk-125-r - Hero Hunk 125 R
Probando modelo: CO2861-tvs-raider-fi-125 - TVS Raider FI 125
Probando modelo: CO1033-bajaj-boxer-ct-100-es - Bajaj Boxer CT 100 ES
Probando modelo: CO1996-hero-hunk-160r-4v - Hero Hunk 160 4V
Probando modelo: CO736-tvs-apache-rtr-160-4v-xc-fi - TVS Apache RTR 160 4V FI ABS
Probando modelo: CO734-tvs-apache-rtr-160-4v-xconnect - TVS Apache RTR 160 4V Xconnect
Probando modelo: CO1480-hero-eco-deluxe-cw-c-2024 - Hero ECO DELUXE CW
Probando modelo: CO569-tvs-neo-nx - TVS NEO NX


In [6]:
print(knowledge_base_final)

CODE = CO731-tvs-raider-125 | TVS Raider 125

 [DATOS BASICOS]
Marca: TVS
Modelo: Raider 125
Año: 2026
Pais: CO
Tipo: ['Urbana']
URL: https://www.galgo.com/co/motos/CO731-tvs-raider-125
    
[SEGMENTO]
Tipo según BD: ['Urbana']
Segmento identificado por usuarios: **'Premium Commuter' / 'Entry Sport'**.
*Nota:* Aunque es una 125cc utilitaria, los usuarios NO la ven como una "moto de trabajo" (tipo GN125 o NKD), sino como una moto de *lifestyle* o primera moto deportiva debido a su estética musculosa y equipamiento tecnológico.

[SENTIMIENTO]
En Colombia, la TVS Raider 125 es percibida como la "reina de la tecnología" en su clase. Existe un orgullo de propiedad muy alto porque **"aparenta ser una moto de 160cc o más"**. El sentimiento general es de satisfacción por obtener "mucho por lo que se paga" (pantalla, modos de manejo, estética), aunque esto se mezcla con una frustración recurrente por la calidad de las llantas de serie y ruidos en los plásticos. Es la moto del "joven que quiere 

In [7]:
with open(f"../src/data/output/gemini/{COUNTRY}-knowledge_base_final.md", "w", encoding="utf-8") as f:
    f.write(knowledge_base_final)